In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
import copy

In [2]:
with open("SavedFeatures/final_feature_vector.json", 'r') as f:
    XY = json.load(f)

In [3]:
step = 50000000
lo = 0
clss = 0
num_classes = 7

while(lo < step*(num_classes-1) + 1):
    hi = lo + step
    lo_txt = lo
    hi_txt = hi
    if(lo_txt == 0):
        lo_txt = '---'
    if(lo_txt == step*(num_classes-1)):
        hi_txt = '+++'
    print('{} to {} :: {}'.format(lo_txt, hi_txt, clss))

    clss += 1
    lo += step

--- to 50000000 :: 0
50000000 to 100000000 :: 1
100000000 to 150000000 :: 2
150000000 to 200000000 :: 3
200000000 to 250000000 :: 4
250000000 to 300000000 :: 5
300000000 to +++ :: 6


In [4]:
len(XY)

8181

In [5]:
def getClass(revenue):
    clss = revenue//step
    return min(clss, num_classes-1)

In [6]:
xrr = []
yrr = []
frq_dst = {}
for i in range(num_classes):
    frq_dst[i] = 0

for xy in XY:
    xrr.append(xy['feature'])
    yrr.append(getClass(xy['target']))
    frq_dst[getClass(xy['target'])] += 1
frq_dst

{0: 5082, 1: 1063, 2: 527, 3: 351, 4: 248, 5: 177, 6: 733}

# Normalize

In [7]:
mxrr = [0]*len(xrr[1])
len(mxrr)

178

In [8]:
for feature in xrr:
    for i in range(len(feature)):
        mxrr[i] = max(mxrr[i], feature[i])

In [9]:
mnrr = copy.deepcopy(mxrr)

for feature in xrr:
    for i in range(len(feature)):
        mnrr[i] = min(mnrr[i], feature[i])

In [10]:
nxrr = [[0]*len(xrr[0])]*len(xrr)

for i in range(len(xrr)):
    for j in range(len(xrr[i])):
        if(mxrr[j]-mnrr[j] != 0):        
            nxrr[i][j]= (xrr[i][j] - mnrr[j]) / (mxrr[j] - mnrr[j])

In [11]:
xrr = np.array(xrr)
nxrr = np.array(nxrr)
yrr = np.array(yrr)

xrr.shape, nxrr.shape, yrr.shape

((8181, 178), (8181, 178), (8181,))

# Train-test split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(nxrr, yrr, test_size=0.30, random_state=17)

# DNN

In [13]:
# mlp for multiclass classification
from numpy import argmax
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
# load the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
df = read_csv(path, header=None)
# split into input and output columns
X, y = df.values[:, :-1], df.values[:, -1]
# ensure all data are floating point values
X = X.astype('float32')
# encode strings to integer

X.shape, y.shape

((150, 4), (150,))

In [14]:
y = LabelEncoder().fit_transform(y)
y.shape

(150,)

In [16]:
len(list(set(y)))

3

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(100, 4) (50, 4) (100,) (50,)


In [17]:
n_features = X_train.shape[1]
# define model
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(3, activation='softmax'))

In [21]:
# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# fit the model
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=1)

Train on 100 samples
Epoch 1/150
100/100 [==============================] - 0s 5ms/sample - loss: 0.4269 - accuracy: 0.8400
Epoch 2/150
100/100 [==============================] - 0s 188us/sample - loss: 0.4211 - accuracy: 0.8900
Epoch 3/150
100/100 [==============================] - 0s 356us/sample - loss: 0.4179 - accuracy: 0.8700
Epoch 4/150
100/100 [==============================] - 0s 311us/sample - loss: 0.4171 - accuracy: 0.8400
Epoch 5/150
100/100 [==============================] - 0s 288us/sample - loss: 0.4153 - accuracy: 0.8400
Epoch 6/150
100/100 [==============================] - 0s 200us/sample - loss: 0.4110 - accuracy: 0.8600
Epoch 7/150
100/100 [==============================] - 0s 370us/sample - loss: 0.4097 - accuracy: 0.8500
Epoch 8/150
100/100 [==============================] - 0s 222us/sample - loss: 0.4061 - accuracy: 0.8600
Epoch 9/150
100/100 [==============================] - 0s 236us/sample - loss: 0.4047 - accuracy: 0.8900
Epoch 10/150
100/100 [==============

In [22]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)

Test Accuracy: 0.980


In [23]:
row = [5.1,3.5,1.4,0.2]
yhat = model.predict([row])
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

Predicted: [[9.9966836e-01 2.2600993e-04 1.0558057e-04]] (class=0)
